In [57]:
#ignore, used to look up OG contents of input file
import os
import pandas as pd
import numpy as np
from scipy.stats import zscore

path = './data/'
filename_read = os.path.join(path,'heart.csv')
df = pd.read_csv(filename_read)


In [58]:
#Useful functions
import os
import pandas as pd

path = "./data/"

filename_read = os.path.join(path, "heart.csv")
df = pd.read_csv(filename_read)

#Randomize the data
np.random.seed(50)
df = df.reindex(np.random.permutation(df.index))
df.reset_index(inplace=True, drop=True)


headers = list(df.columns.values)
fields = []

for field in headers:
    fields.append({
        'data Val' : field,
        'mean' : df[field].mean(),
        'var' : df[field].var(),
        'sdev' : df[field].std()
    })
    
# Learned from Jeff Heaton: 
# https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class03_tensor_flow.ipynb
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

    


In [81]:
#Classification Neural Network
import os
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import requests 
import io 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from keras.models import Sequential 
from keras.layers.core import Dense, Activation 
from keras.callbacks import EarlyStopping

path = "./data/"

filename_read = os.path.join(path, "heart.csv") 
df = pd.read_csv(filename_read)

#one-hot coding for catagory classification data 
one_hot = pd.get_dummies(df['sex']) 
df = df.drop('sex', axis = 1) 
df = df.join(one_hot) 
df.rename(columns = { 0: "female",
                     1: "male"}, 
          inplace = True) 
one_hot = pd.get_dummies(df['fbs']) 
df = df.drop('fbs', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "fbs-no",
                     1: "fbs-yes"}, 
          inplace = True) 

one_hot = pd.get_dummies(df['exang'])
df = df.drop('exang', axis = 1) 
df = df.join(one_hot)
df.rename(columns = { 0: "exang-no",
                     1: "exang-yes"},
          inplace = True)

#Normalize the real number data
df['age'] = zscore(df['age']) 
df['cp'] = zscore(df['cp']) 
df['trestbps'] = zscore(df['trestbps']) 
df['chol'] = zscore(df['chol']) 
df['restecg'] = zscore(df['restecg']) 
df['age'] = zscore(df['age']) 
df['thalach'] = zscore(df['thalach']) 
df['oldpeak'] = zscore(df['oldpeak']) 
df['slope'] = zscore(df['slope']) 
df['ca'] = zscore(df['ca']) 
df['thal'] = zscore(df['thal'])

df = df.reindex(np.random.permutation(df.index))#shuffle the data 
df.reset_index(inplace=True, drop=True)



#Pre-segment the data
mask = np.random.rand(len(df)) < 0.85
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])

# Use the to_xy() to convert the pre-segmented data into x,y corridnates used by tensorflow
x,y = to_xy(trainDF, "target")
x1,y1 = to_xy(validationDF, "target")

#split into train/test 
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.15, random_state=42)#Train on 70% Tes on 30

model = Sequential()
model.add(Dense(13, input_dim=x.shape[1], activation='relu'))#IHidden layer1
model.add(Dense(7,activation='relu')) #Hidden Layer
model.add(Dense(y.shape[1],activation='softmax')) #Output Layer
model.compile(loss='categorical_crossentropy', optimizer='adam') 
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,patience=5, verbose=1, mode='auto')#Stop, when improvments arn't made for 5 turns              

model.fit(x,y, validation_data=(x_test,y_test),callbacks=[monitor], verbose=2,epochs=1000)

Train on 259 samples, validate on 39 samples
Epoch 1/1000
 - 1s - loss: 1.2377 - val_loss: 1.6331
Epoch 2/1000
 - 0s - loss: 1.1336 - val_loss: 1.4726
Epoch 3/1000
 - 0s - loss: 1.0412 - val_loss: 1.3386
Epoch 4/1000
 - 0s - loss: 0.9617 - val_loss: 1.2237
Epoch 5/1000
 - 0s - loss: 0.8955 - val_loss: 1.1198
Epoch 6/1000
 - 0s - loss: 0.8354 - val_loss: 1.0243
Epoch 7/1000
 - 0s - loss: 0.7803 - val_loss: 0.9328
Epoch 8/1000
 - 0s - loss: 0.7303 - val_loss: 0.8511
Epoch 9/1000
 - 0s - loss: 0.6876 - val_loss: 0.7883
Epoch 10/1000
 - 0s - loss: 0.6510 - val_loss: 0.7277
Epoch 11/1000
 - 0s - loss: 0.6192 - val_loss: 0.6703
Epoch 12/1000
 - 0s - loss: 0.5894 - val_loss: 0.6240
Epoch 13/1000
 - 0s - loss: 0.5658 - val_loss: 0.5832
Epoch 14/1000
 - 0s - loss: 0.5430 - val_loss: 0.5457
Epoch 15/1000
 - 0s - loss: 0.5235 - val_loss: 0.5126
Epoch 16/1000
 - 0s - loss: 0.5071 - val_loss: 0.4807
Epoch 17/1000
 - 0s - loss: 0.4906 - val_loss: 0.4549
Epoch 18/1000
 - 0s - loss: 0.4774 - val_loss:

In [82]:
from sklearn import metrics
import tensorflow as tf

#Run the model on the test sample
pred1 = model.predict(x_test)
pred1 = np.argmax(pred1, axis=1)
print("Shape: {}".format(pred1.shape))
print(pred1)

y_compare = np.argmax(y_test, axis=1)
score = metrics.accuracy_score(y_compare, pred1)
print("Accuracy: {}".format(score))

Shape: (39,)
[0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0
 1 0]
Accuracy: 0.9487179487179487


In [83]:
#Run model on validation sample
validation_pred = model.predict(x1)
predicted_hd = np.argmax(validation_pred, axis=1)

expected_hd = np.argmax(y1, axis=1)

score = metrics.accuracy_score(expected_hd, predicted_hd)
comparison = (predicted_hd == expected_hd)
print("Predictions: {}".format(predicted_hd))
print("Expected: {}".format(expected_hd))
print("Comparison: {}".format(comparison))
print("Accuracy: {}".format(score))

Predictions: [0 0 0 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0
 0 1 0 1 1 1 1]
Expected: [0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 0
 0 0 0 1 1 0 1]
Comparison: [ True  True False False  True  True  True  True  True  True  True  True
 False False  True  True  True  True False  True  True False  True  True
 False  True  True  True  True  True  True  True  True  True False  True
  True  True False  True  True  True False  True]
Accuracy: 0.7727272727272727


In [35]:
#Calculate log loss (Inaccuracy of predictions) 
from IPython.display import display

# Dont display numpy in SI notation
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
display(pred[0]*100)

print("As percent probability")
display(pred[0:5]*100)

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions


array([98.7615,  1.2385], dtype=float32)

As percent probability


array([[98.7615,  1.2385],
       [ 0.1439, 99.8561],
       [99.9829,  0.0171],
       [86.5704, 13.4296],
       [ 4.7954, 95.2046]], dtype=float32)

Log loss score: 0.040459821792426034
